In [13]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(np.sqrt(rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(np.sqrt(rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-(mockForcastDay+forecastDay)] & np.log
    ds = rawArrayDatas[0][:-(mockForcastDay+forecastDay)]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-(mockForcastDay+forecastDay)]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of (mockForcastDay+forecastDay)  rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]
    testY= rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출       
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'day')
            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')
    
            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    print('LSTM realforecast :',realForecastDictionary['LSTM'])
    print('Bayseian realforecast :',realForecastDictionary['Bayseian'] ) 
    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
#         listedLogPredict=test_predict[-1].tolist()
#     return [np.exp(y) for y in listedLogPredict]
    return np.square(test_predict[-1]).tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM testforecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian testforecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm


In [14]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [15]:
columns=['ds','y']
df=pd.read_table('webMonth36.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.2)
print(len(df), forecastDay)


36 7


In [16]:
df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

C:\Users\User\Anaconda3\envs\tensorflow35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
rawArrayDatas

[['2013-02-01',
  '2013-03-01',
  '2013-04-01',
  '2013-05-01',
  '2013-06-01',
  '2013-07-01',
  '2013-08-01',
  '2013-09-01',
  '2013-10-01',
  '2013-11-01',
  '2013-12-01',
  '2014-01-01',
  '2014-02-01',
  '2014-03-01',
  '2014-04-01',
  '2014-05-01',
  '2014-06-01',
  '2014-07-01',
  '2014-08-01',
  '2014-09-01',
  '2014-10-01',
  '2014-11-01',
  '2014-12-01',
  '2015-01-01',
  '2015-02-01',
  '2015-03-01',
  '2015-04-01',
  '2015-05-01',
  '2015-06-01',
  '2015-07-01',
  '2015-08-01',
  '2015-09-01',
  '2015-10-01',
  '2015-11-01',
  '2015-12-01',
  '2016-01-01'],
 [4432.5,
  2776.5161290000001,
  2286.6333329999998,
  1913.451613,
  1650.9333329999999,
  2040.0333329999999,
  2734.0645159999999,
  13740.266669999999,
  10564.35484,
  7608.3999999999996,
  8706.1935480000011,
  26366.56667,
  24504.89286,
  2169.0322579999997,
  1879.0,
  2449.5483870000003,
  1487.9000000000001,
  1903.580645,
  3121.333333,
  8773.6333329999998,
  9429.2903230000011,
  4370.5666670000001,
  412

In [18]:
answer=LearningModuleRunner(rawArrayDatas,0,forecastDay,'month') #0은 processId

[step: 0] loss: 0.19120542705059052
[step: 1] loss: 0.05840928480029106
[step: 2] loss: 0.016061391681432724
[step: 3] loss: 0.02930806204676628
[step: 4] loss: 0.0554005391895771
[step: 5] loss: 0.06284425407648087
[step: 6] loss: 0.05171339586377144
[step: 7] loss: 0.03442814573645592
[step: 8] loss: 0.020501837134361267
[step: 9] loss: 0.013794281519949436
[step: 10] loss: 0.013762188144028187
[step: 11] loss: 0.017730500549077988
[step: 12] loss: 0.022702645510435104
[step: 13] loss: 0.026286548003554344
[step: 14] loss: 0.02716263383626938
[step: 15] loss: 0.025298263877630234
[step: 16] loss: 0.02170492149889469
[step: 17] loss: 0.017786119133234024
[step: 18] loss: 0.014691222459077835
[step: 19] loss: 0.013004560023546219
[step: 20] loss: 0.012774770148098469
[step: 21] loss: 0.013669103384017944
[step: 22] loss: 0.015108455903828144
[step: 23] loss: 0.016425661742687225
[step: 24] loss: 0.01709100790321827
[step: 25] loss: 0.016904251649975777
[step: 26] loss: 0.01601881533861

[step: 267] loss: 0.004575403407216072
[step: 268] loss: 0.004722296725958586
[step: 269] loss: 0.004670402500778437
[step: 270] loss: 0.00451134005561471
[step: 271] loss: 0.004517870023846626
[step: 272] loss: 0.004615289159119129
[step: 273] loss: 0.0045782420784235
[step: 274] loss: 0.004471833351999521
[step: 275] loss: 0.004476219415664673
[step: 276] loss: 0.0045371465384960175
[step: 277] loss: 0.004510143771767616
[step: 278] loss: 0.004435969050973654
[step: 279] loss: 0.004432471003383398
[step: 280] loss: 0.004471724387258291
[step: 281] loss: 0.004458783660084009
[step: 282] loss: 0.0044059669598937035
[step: 283] loss: 0.004388208035379648
[step: 284] loss: 0.004410190507769585
[step: 285] loss: 0.004414965398609638
[step: 286] loss: 0.004383270163089037
[step: 287] loss: 0.00435333838686347
[step: 288] loss: 0.004352869000285864
[step: 289] loss: 0.004364342428743839
[step: 290] loss: 0.004359100945293903
[step: 291] loss: 0.004335496108978987
[step: 292] loss: 0.0043145

[step: 527] loss: 0.00322248344309628
[step: 528] loss: 0.003219187492504716
[step: 529] loss: 0.0032155094668269157
[step: 530] loss: 0.0032133222557604313
[step: 531] loss: 0.0032116922084242105
[step: 532] loss: 0.0032088798470795155
[step: 533] loss: 0.003205514047294855
[step: 534] loss: 0.003202760824933648
[step: 535] loss: 0.0032004450913518667
[step: 536] loss: 0.0031981514766812325
[step: 537] loss: 0.0031958038453012705
[step: 538] loss: 0.003193062264472246
[step: 539] loss: 0.0031900391913950443
[step: 540] loss: 0.003187308320775628
[step: 541] loss: 0.003184964880347252
[step: 542] loss: 0.003182652872055769
[step: 543] loss: 0.0031802731100469828
[step: 544] loss: 0.003177881008014083
[step: 545] loss: 0.003175352932885289
[step: 546] loss: 0.0031726632732897997
[step: 547] loss: 0.0031700138933956623
[step: 548] loss: 0.0031674408819526434
[step: 549] loss: 0.0031648457515984774
[step: 550] loss: 0.0031622580718249083
[step: 551] loss: 0.0031597481574863195
[step: 552]

[step: 794] loss: 0.006230633240193129
[step: 795] loss: 0.0036982540041208267
[step: 796] loss: 0.0048334551975131035
[step: 797] loss: 0.004373893607407808
[step: 798] loss: 0.003587396815419197
[step: 799] loss: 0.004801964852958918
[step: 800] loss: 0.0031438178848475218
[step: 801] loss: 0.004352217074483633
[step: 802] loss: 0.0036400111857801676
[step: 803] loss: 0.0034301637206226587
[step: 804] loss: 0.004079741425812244
[step: 805] loss: 0.0031804032623767853
[step: 806] loss: 0.003676473395898938
[step: 807] loss: 0.00359707442112267
[step: 808] loss: 0.0031696646474301815
[step: 809] loss: 0.003627506084740162
[step: 810] loss: 0.0033032565843313932
[step: 811] loss: 0.00321705499663949
[step: 812] loss: 0.003488450078293681
[step: 813] loss: 0.0031481378246098757
[step: 814] loss: 0.0032478433568030596
[step: 815] loss: 0.003334619104862213
[step: 816] loss: 0.003081402275711298
[step: 817] loss: 0.0032477150671184063
[step: 818] loss: 0.0031981882639229298
[step: 819] los

testY is:  [4122.7096770000007, 7644.7419349999991, 4147.0740740000001, 1799.7096770000001, 1405.5, 1648.193548, 1090.5666670000001]


LSTM testforecast : [11501.126953125, 35231.7890625, 23929.568359375, 3238.9677734375, 3742.501953125, 2987.68798828125, 256.23822021484375] 
@@@@@LSTM rmse:  13184.7134547
Bayseian testforecast : [734.03919080076446, 1894.5132882945629, 13344240.389051739, 65.965064237998888, 156.69537789750424, 2934.0792310475513, 467.71113202047519] 
@@@@@Bayseian rmse:  5042082.06438


LSTM WON!!!!!!
[step: 0] loss: 0.4193812906742096
[step: 1] loss: 0.12354608625173569
[step: 2] loss: 0.026315681636333466
[step: 3] loss: 0.05264170840382576
[step: 4] loss: 0.11139579117298126
[step: 5] loss: 0.130569189786911
[step: 6] loss: 0.10772176831960678
[step: 7] loss: 0.06949283927679062
[step: 8] loss: 0.037501294165849686
[step: 9] loss: 0.021277157589793205
[step: 10] loss: 0.02042325586080551
[step: 11] loss: 0.02906809188425541
[step: 12] loss: 0.040254343301057816
[s

[step: 223] loss: 0.010085422545671463
[step: 224] loss: 0.010059383697807789
[step: 225] loss: 0.01004967000335455
[step: 226] loss: 0.010030255652964115
[step: 227] loss: 0.010006063617765903
[step: 228] loss: 0.00999431498348713
[step: 229] loss: 0.009978371672332287
[step: 230] loss: 0.009956086054444313
[step: 231] loss: 0.009942302480340004
[step: 232] loss: 0.009928961284458637
[step: 233] loss: 0.00990918930619955
[step: 234] loss: 0.009893665090203285
[step: 235] loss: 0.009881586767733097
[step: 236] loss: 0.009864822961390018
[step: 237] loss: 0.009848405607044697
[step: 238] loss: 0.009836221113801003
[step: 239] loss: 0.009822247549891472
[step: 240] loss: 0.009806251153349876
[step: 241] loss: 0.00979317631572485
[step: 242] loss: 0.009780971333384514
[step: 243] loss: 0.009766547940671444
[step: 244] loss: 0.009752750396728516
[step: 245] loss: 0.009740972891449928
[step: 246] loss: 0.009728393517434597
[step: 247] loss: 0.009714864194393158
[step: 248] loss: 0.009702671

[step: 442] loss: 0.008488179184496403
[step: 443] loss: 0.008483151905238628
[step: 444] loss: 0.008478214032948017
[step: 445] loss: 0.008473352529108524
[step: 446] loss: 0.008468512445688248
[step: 447] loss: 0.00846363790333271
[step: 448] loss: 0.0084586963057518
[step: 449] loss: 0.008453711867332458
[step: 450] loss: 0.008448710665106773
[step: 451] loss: 0.008443720638751984
[step: 452] loss: 0.008438758552074432
[step: 453] loss: 0.008433817885816097
[step: 454] loss: 0.00842889305204153
[step: 455] loss: 0.008423960767686367
[step: 456] loss: 0.008419019170105457
[step: 457] loss: 0.00841408222913742
[step: 458] loss: 0.008409127593040466
[step: 459] loss: 0.008404169231653214
[step: 460] loss: 0.00839921459555626
[step: 461] loss: 0.00839428324252367
[step: 462] loss: 0.008389382623136044
[step: 463] loss: 0.008384553715586662
[step: 464] loss: 0.008379857055842876
[step: 465] loss: 0.008375419303774834
[step: 466] loss: 0.008371519856154919
[step: 467] loss: 0.008368720300

[step: 665] loss: 0.007576595991849899
[step: 666] loss: 0.007570535410195589
[step: 667] loss: 0.007564891595393419
[step: 668] loss: 0.007560122292488813
[step: 669] loss: 0.007555777672678232
[step: 670] loss: 0.007551131770014763
[step: 671] loss: 0.007545880042016506
[step: 672] loss: 0.007540284190326929
[step: 673] loss: 0.007534767035394907
[step: 674] loss: 0.007529598195105791
[step: 675] loss: 0.007524752523750067
[step: 676] loss: 0.007520013488829136
[step: 677] loss: 0.007515172008424997
[step: 678] loss: 0.0075101349502801895
[step: 679] loss: 0.007504911627620459
[step: 680] loss: 0.007499587256461382
[step: 681] loss: 0.007494247518479824
[step: 682] loss: 0.007488942705094814
[step: 683] loss: 0.00748369051143527
[step: 684] loss: 0.007478495594114065
[step: 685] loss: 0.007473332807421684
[step: 686] loss: 0.00746821379289031
[step: 687] loss: 0.007463097106665373
[step: 688] loss: 0.007458019070327282
[step: 689] loss: 0.007452984806150198
[step: 690] loss: 0.007448

[step: 893] loss: 0.006985949352383614
[step: 894] loss: 0.007158966735005379
[step: 895] loss: 0.006976468022912741
[step: 896] loss: 0.007059192750602961
[step: 897] loss: 0.006985069252550602
[step: 898] loss: 0.006972259841859341
[step: 899] loss: 0.006997063290327787
[step: 900] loss: 0.006910810247063637
[step: 901] loss: 0.006994585040956736
[step: 902] loss: 0.0068842563778162
[step: 903] loss: 0.006966775748878717
[step: 904] loss: 0.0068894000723958015
[step: 905] loss: 0.006922387983649969
[step: 906] loss: 0.006907110568135977
[step: 907] loss: 0.006880789063870907
[step: 908] loss: 0.0069136787205934525
[step: 909] loss: 0.006859442684799433
[step: 910] loss: 0.006896985229104757
[step: 911] loss: 0.0068593923933804035
[step: 912] loss: 0.006865113507956266
[step: 913] loss: 0.006865036208182573
[step: 914] loss: 0.006837137974798679
[step: 915] loss: 0.006858511827886105
[step: 916] loss: 0.006825403776019812
[step: 917] loss: 0.006836588494479656
[step: 918] loss: 0.0068

In [19]:
len(answer)

7

In [20]:
answer

[11564.00390625,
 19378.8515625,
 12206.912109375,
 3621.99755859375,
 6344.431640625,
 4044.051025390625,
 1171.41552734375]

In [21]:
len(rawArrayDatas[1])

36